In [1]:
# !pip install nltk
# import nltk
# nltk.download('twitter_samples')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [2]:
ANNOTATIONS_PATH = "claws_classifier/annotations.csv"
APRIL_TWEETS_CLAWS_CLASSIFIED_DIR_PATH = '/home/dhruv/Downloads/kaggle-covid-en-tweets-april_classified_claws/'

In [3]:
import pickle, os
import pandas as pd 

train_data = pd.read_csv(ANNOTATIONS_PATH) 

test_data = []
for csv_file in os.listdir(APRIL_TWEETS_CLAWS_CLASSIFIED_DIR_PATH):
    df = pd.read_csv(APRIL_TWEETS_CLAWS_CLASSIFIED_DIR_PATH + csv_file, usecols = ['text','Label'],index_col=None, header=0)
    test_data.append(df)
    
test_data = pd.concat(test_data, axis=0, ignore_index=True)
test_data = test_data.rename(columns={'text': 'Text'})

/home/dhruv/env3_bertviz/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

def get_tweet_tokens(tweet_text):
    return TweetTokenizer().tokenize(tweet_text)

def get_dataset(train=False,test=False):
    if train:
        counterhate_tweets = train_data[train_data['Label'] == 'Counterhate']['Text']
        hate_tweets = train_data[train_data['Label'] == 'Hate']['Text']
    elif test:
        counterhate_tweets = test_data[test_data['Label'] == 'Counterhate']['Text']
        hate_tweets = test_data[test_data['Label'] == 'Hate']['Text']

    counterhate_tweet_tokens = counterhate_tweets.apply(get_tweet_tokens)
    hate_tweet_tokens = hate_tweets.apply(get_tweet_tokens)

    stop_words = stopwords.words('english')

    counterhate_cleaned_tokens_list = []
    hate_cleaned_tokens_list = []

    for tokens in counterhate_tweet_tokens.tolist():
        counterhate_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in hate_tweet_tokens.tolist():
        hate_cleaned_tokens_list.append(remove_noise(tokens, stop_words)) 


    counterhate_tokens_for_model = get_tweets_for_model(counterhate_cleaned_tokens_list)
    hate_tokens_for_model = get_tweets_for_model(hate_cleaned_tokens_list)


    counterhate_dataset = [(tweet_dict, "Counterhate")
                         for tweet_dict in counterhate_tokens_for_model]

    hate_dataset = [(tweet_dict, "Hate")
                         for tweet_dict in hate_tokens_for_model]

    dataset = counterhate_dataset + hate_dataset

    random.shuffle(dataset)
    return dataset


def classify_tweet(tweet_text):
    custom_tokens = remove_noise(word_tokenize(tweet_text))
    return classifier.classify(dict([token, True] for token in custom_tokens))

In [5]:
train_data = get_dataset(train=True)
test_data = get_dataset(test=True)
    
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.8570035707954516
Most Informative Features
           #iamnotavirus = True           Counte : Hate   =     29.4 : 1.0
                   stand = True           Counte : Hate   =     23.3 : 1.0
         #racismisavirus = True           Counte : Hate   =     21.6 : 1.0
                 #racism = True           Counte : Hate   =     18.2 : 1.0
                 calling = True           Counte : Hate   =     15.7 : 1.0
                     ccp = True             Hate : Counte =     15.4 : 1.0
                violence = True           Counte : Hate   =     13.2 : 1.0
                  racism = True           Counte : Hate   =     13.2 : 1.0
                   asian = True           Counte : Hate   =     12.1 : 1.0
                  target = True           Counte : Hate   =     11.9 : 1.0
None
